In [2]:
import numpy as np
import os
from keras.models import Model
import pandas as pd
import os
import matplotlib.pyplot
%matplotlib inline
import librosa
import lib.util as ut
import tensorflow as tf

/home/alberto/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the model we want to make inferences with

In [3]:
model_path='my_model.h5'

## To divide audio files. When making inferences over whole audio files, we pass interval_step=sample_rate/2 to produce 50% overlapping chunks of 3 second audio

In [4]:
def divide_audio_file(path, intervals_seconds, interval_step, sample_rate=8000):
    """

    """
    # loads file and converts to the specified sample rate    
    audio, fs = librosa.load(path, sample_rate)
    
    audio_array = []
    for i in range(0, audio.shape[0], interval_step):   
        interval = audio[i:i+sample_rate*intervals_seconds]
#         print("interval from {} to {}".format(i, i+sample_rate*intervals_seconds))
        
        # if the last interval is shorter han the interval in seconds we define we are going to ignore it
        if interval.shape[0] < sample_rate*intervals_seconds:
            break
        else:
            if (not ut.is_silence(interval,thresold_samples=0.70)):
                audio_array.append(interval)
            else:
                print("Omitting chunk with silences in file {}".format(path))

    return np.array(audio_array)

## Like the original create_tf_records but creating numpy array's instead

In [5]:
def get_chunks_from_raw(audio_path, file_names_df, num_samples):
    
    x_data = []
    labels = []
    tf_record_files = 1  
    for file in os.listdir(audio_path):

        file_path = audio_path + "/" + file
        short_name = file.split(".")[0]

        # if the file is in the dataframe with the file names(train or test) we divide it, if not we ignore
        if short_name in file_names_df["0"].values:
            print("reading file {}".format(file))

            divided_file =  divide_audio_file(file_path, intervals_seconds, sample_rate)

            file_label = file_names_df[file_names_df["0"] == short_name]["class"].values[0]
            labels_array = np.ones(divided_file.shape[0]) * file_label
            
            x_data.extend(divided_file)
            labels.extend(labels_array)            
            
            if len(x_data) > num_samples:
                tf_record_files += 1
                break;
                        
        else:
            print("file {} not in the dataframe".format(file))
       
                 
    return (x_data,labels)



## makes inference of a whole audio by dividing it into overlapping (50%) chunks

In [6]:
def predict_one_audio(file_name,label):
    file_path = audio_path + "/" + file_name
    audio_chunks=divide_audio_file(file_path, intervals_seconds, int(sample_rate*1.5))
    audio_chunks = audio_chunks.reshape([audio_chunks.shape[0],intervals_seconds * sample_rate,1])
    labels_array = np.ones(audio_chunks.shape[0]) * label
    predictions = model.predict(audio_chunks)
#     print(labels_array.shape)
#     print(audio_chunks.shape)
#     vc = pd.DataFrame(predictions)[0].apply(lambda x: 0 if x < 0.5 else 1).value_counts()
#     if (vc[0]<vc[1]):
#         res = 1
#     else:
#         res = 0
        
    return (predictions)
    
    

## Get string labels from numeric ones to build a confusion matrix 

In [7]:
def getLabels(x):
    if (x>=0.5):
        label = "CY"
    else:
        label = "CN"
    return(label)
    


## Begin of the execution

In [8]:
from keras.models import load_model
model = load_model(model_path)
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/alberto/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:74: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 24000, 1)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 24000, 64)         8320      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3000, 64)          0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 3000, 64, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3000, 64, 96)      374880    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 375, 64, 96)       0         
________________

In [9]:
audio_path = "./data/audiodata"
labels_file = "./data/conflictlevel.csv"

intervals_seconds = 3
sample_rate = 8000

In [10]:
labels_df = pd.read_csv(labels_file, header=None)
labels_df["class"] = labels_df[1].apply(lambda x: 0 if x < 0 else 1)



In [11]:
# train_df = pd.read_csv("./data/train_files.csv")
test_df = pd.read_csv("./data/test_files.csv")
# validation_df = pd.read_csv("./data/validation_files.csv")

## Prediction on test set

In [12]:
test_df.head()

,0,1,class
0,08-01-23_780_810,-5.3,0
1,06-09-06_1470_1500,4.6,1
2,08-01-09_1650_1680,-0.2,0
3,07-01-31_2040_2070,3.5,1
4,08-01-23_990_1020,-2.5,0


In [39]:
predictions_list = []

for file, cl in test_df[["0", "class"]].values[:5]:
    pred = predict_one_audio(file + ".wav",cl)
    
    print("predicting for file {}".format(file + ".wav"))
    predictions_list.append([i for sublist in pred for i in sublist])
    

predicting for file 08-01-23_780_810.wav
predicting for file 06-09-06_1470_1500.wav
predicting for file 08-01-09_1650_1680.wav
predicting for file 07-01-31_2040_2070.wav
predicting for file 08-01-23_990_1020.wav


In [40]:
pred_df = pd.DataFrame({"prediction": predictions_list})

In [55]:
# test_df = test_df.iloc[:5]

In [56]:
final_df = test_df.join(pred_df)

In [68]:
final_df["avg"] = final_df["prediction"].apply(lambda x: np.mean(x))
final_df["stdv"] = final_df["prediction"].apply(lambda x: np.std(x))
final_df["positive"] = final_df["prediction"].apply(lambda x: sum(i >= 0.5 for i in x))
final_df["negative"] = final_df["prediction"].apply(lambda x: sum(i < 0.5 for i in x))
final_df["majority"] = final_df.apply(lambda row: 1 if row["positive"] >= row["negative"] else 0, axis=1)

In [69]:
final_df.head()

,0,1,class,prediction,avg,stdv,positive,negative,majority
0,08-01-23_780_810,-5.3,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0,19,0
1,06-09-06_1470_1500,4.6,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0,19,0
2,08-01-09_1650_1680,-0.2,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0,19,0
3,07-01-31_2040_2070,3.5,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0,19,0
4,08-01-23_990_1020,-2.5,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0,19,0


In [ ]:
train_df.to_csv("./data/train_files.csv", index=False)

In [61]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y_arr)

In [62]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

             precision    recall  f1-score   support

         CN       0.60      1.00      0.75         3
         CY       0.00      0.00      0.00         2

avg / total       0.36      0.60      0.45         5



/home/alberto/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
